# In this file we fit our model with the real data.

In [7]:
from datetime import datetime
import random

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import norm, chi2

from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox  

from Garch import GARCH

#### Define helper functions 

In [8]:
def adf_test(series):
    result = adfuller(series, autolag='AIC')  
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"   {key}: {value}")

    if result[1] <= 0.05:
        print(" The series is stationary (reject H0)")
    else:
        print(" The series is non-stationary (fail to reject H0)")

In [9]:
def generate_ar_data(ar_coef, T):
    res = [0]
    for t in range(1,T):
        res.append(ar_coef * res[-1] + random.gauss(0, 1))

    return res

In [10]:
def likelihood_ratio_test(ll_null: float, ll_alternative: float, df: int):
    if df <= 0:
        raise ValueError("Degrees of freedom must be positive.")
    if ll_alternative < ll_null:
        raise ValueError("ll_null should be smaller than ll_alternative.")
    
    # Compute test statistic
    lr_stat = 2 * (ll_alternative - ll_null)
    
    # Compute p-value
    p_value = 1 - chi2.cdf(lr_stat, df)
    
    return p_value

## 1. Load Data
1. S&P500 data
2. Sentiment data (consisting of mean Positive, Negative and Neutral sentiment per day)
3. VIX (for comparison purpose)

In [11]:
start = '2019-10-01'
end = '2024-09-27'

In [12]:
# Load S&P data
prices_df = pd.read_csv('../data/spy_prices.csv')
prices_df.index = pd.to_datetime(prices_df['Date'], format='%Y-%m-%d %H:%M:%S%z', utc=True).dt.date

# Load Sentiment data
sentiment_df = pd.read_csv('../data/nyt_sentiment.csv')
sentiment_df.index = pd.DatetimeIndex(sentiment_df['adjusted_date'])

# Load US interest rate data
t_rates_df = pd.read_csv('../data/daily-treasury-rates.csv')
t_rates_df.index = pd.DatetimeIndex(t_rates_df['Date'])
t_rates_df = t_rates_df.drop(columns=['Date','Unnamed: 11',	'Unnamed: 12', '26 WEEKS BANK DISCOUNT', '26 WEEKS COUPON EQUIVALENT'])

DateParseError: Unknown datetime string format, unable to parse: adjusted_date, at position 1497

In [13]:
# Merge all into one dataframe
#vix.index = pd.DatetimeIndex(vix.index.tz_localize(None))
data_with_sentiment = prices_df.join(sentiment_df, how='inner')#.join(vix[['VIX Close']], how='left')
data_with_sentiment = data_with_sentiment.drop(columns=['Date', 'adjusted_date'])

data_with_sentiment['neg_sentiment_lag1'] = data_with_sentiment['mean_neg_sentiment'].shift(1)
data_with_sentiment['neg_sentiment_diff'] = data_with_sentiment['mean_neg_sentiment'] - data_with_sentiment['neg_sentiment_lag1']
data_with_sentiment = data_with_sentiment.dropna()

data_with_sentiment = data_with_sentiment.join(t_rates_df)
# Fill missing i/r data. Missing data is sparse, but might have to investigate.
data_with_sentiment = data_with_sentiment.ffill()

log_returns = data_with_sentiment['log_returns']

data_with_sentiment.head()

NameError: name 't_rates_df' is not defined

## 2. Fit a baseline GARCH(1,1) model without the exogenous term. 

In [7]:
garch_baseline = GARCH(p=1, q=1, z=0, verbose=True)
garch_baseline.train(100*log_returns)

garch_baseline.summary()

Optimising...
Optimising finished in 1.946s
{'omega': 0.03623706316334024, 'alpha': 0.15244767730571898, 'beta': 0.8239142716887397}


,coef,std err,t,P>|t|
omega,0.036237,0.009765,3.710942,1.078717e-04
alpha,0.152448,0.023169,6.579677,3.483829e-11
beta,0.823914,0.024032,34.284148,1.701965e-181


In [8]:
baseline_log_like = garch_baseline.loglikelihood
print(f"Log likelihood: {baseline_log_like:.3f}")

Log likelihood: -1766.938


## 2. Fit model with the exogenous variables. 

In [9]:
exo_sentiment = data_with_sentiment[['mean_pos_sentiment',	'mean_neg_sentiment', 'mean_neutral_sentiment']].to_numpy()

#### Try alternative data using PCA since the three sentiments features are correlated.  

In [10]:
# ! pip install scikit-learn
# from sklearn.decomposition import PCA

# pca = PCA(n_components=1)
# pca_exo_sentiment = pca.fit_transform(exo_sentiment)

You should consider upgrading via the 'd:\programming\sentiment-analysis-volatility-forecasting\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
garch_with_sentiment = GARCH(p=1, q=1, z=1, verbose=True)
garch_with_sentiment.train(100*log_returns, x=exo_sentiment)

garch_with_sentiment.summary()

Optimising...
Optimising finished in 13.238s
{'omega': 1.1046467547436256e-06, 'alpha': 0.1650554008369235, 'beta': 0.8032110131541966, 'gamma': array([[1.11526057e-11, 1.81513449e-01, 3.31085876e-02]])}


,coef,std err,t,P>|t|
omega,1.104647e-06,0.107638,1.026259e-05,4.999959e-01
alpha,1.650554e-01,0.026350,6.263935e+00,2.593112e-10
beta,8.032110e-01,0.029499,2.722843e+01,2.393297e-128
gamma[0],1.115261e-11,0.595625,1.872420e-11,5.000000e-01
gamma[1],1.815134e-01,0.236359,7.679552e-01,2.213309e-01
gamma[2],3.310859e-02,0.255889,1.293863e-01,4.485366e-01


In [12]:
with_sentiment_log_like = garch_with_sentiment.loglikelihood
print(f"Log likelihood: {with_sentiment_log_like:.3f}")

Log likelihood: -1765.046


## 2.2 Fit with only negative sentiment.

In [13]:
neg_sentiment = data_with_sentiment['mean_neg_sentiment'].to_numpy()#exo_sentiment[:, [1]]
#neg_sentiment_normalised = (neg_sentiment - np.mean(neg_sentiment)) / np.var(neg_sentiment)

#### Run Ljung-Box test (default lags=10)


In [14]:
#result = acorr_ljungbox(neg_sentiment, lags=[x for x in range(11)], return_df=True)
#print(result)

#### Test exogenous data for stationarity


In [15]:
print("ADF Test for X:")
adf_test(neg_sentiment)

ADF Test for X:
ADF Statistic: -4.243249057941488
p-value: 0.0005562151981538337
Critical Values:
   1%: -3.4357748759345768
   5%: -2.8639355566269677
   10%: -2.5680454348767596
 The series is stationary (reject H0)


In [16]:
garch_with_neg_sentiment = GARCH(p=1, q=1, z=1, verbose=True)
garch_with_neg_sentiment.train(100*log_returns, x=neg_sentiment)

Optimising...
Optimising finished in 5.050s
{'omega': 0.003168880615102086, 'alpha': 0.16563081648344297, 'beta': 0.8018300421493844, 'gamma': array([[0.19929919]])}


In [18]:
garch_with_neg_sentiment.summary()

,coef,std err,t,P>|t|
omega,0.003169,0.020561,0.154122,4.387694e-01
alpha,0.165631,0.026193,6.323477,1.788038e-10
beta,0.801830,0.029225,27.436576,6.953446e-130
gamma[0],0.199299,0.119357,1.669773,4.760984e-02


In [19]:
with_neg_sentiment_log_like = garch_with_sentiment.loglikelihood
print(f"Log likelihood: {with_neg_sentiment_log_like:.3f}")

Log likelihood: -1765.046


## 2.3 Fit with other data.

In [32]:
rf_data = data_with_sentiment[['4 WEEKS BANK DISCOUNT', 'mean_neg_sentiment']]   # '8 WEEKS BANK DISCOUNT',

In [33]:
garch_with_neg_sentiment = GARCH(p=1, q=1, z=1, verbose=True)
garch_with_neg_sentiment.train(100*log_returns, x=rf_data)

Optimising...
Optimising finished in 10.487s
{'omega': 0.0015569817707142767, 'alpha': 0.16578033629485042, 'beta': 0.8014779205806908, 'gamma': array([[6.34962711e-07, 2.07599812e-01]])}


In [34]:
garch_with_neg_sentiment.summary()

,coef,std err,t,P>|t|
omega,1.556982e-03,0.011936,0.130439,4.481201e-01
alpha,1.657803e-01,0.032517,5.098203,1.984559e-07
beta,8.014779e-01,0.034868,22.986158,8.475967e-98
gamma[0],6.349627e-07,0.000551,0.001152,4.995407e-01
gamma[1],2.075998e-01,0.053287,3.895872,5.156809e-05


In [23]:
with_neg_sentiment_log_like = garch_with_neg_sentiment.loglikelihood
print(f"Log likelihood: {with_neg_sentiment_log_like:.3f}")

Log likelihood: -1766.900


In [54]:
likelihood_ratio_test(garch_baseline.loglikelihood, 
                      garch_with_neg_sentiment.loglikelihood, 2)

0.9625447383437402

---

In [15]:
def mse(actual, pred):
    return np.sum((actual-pred) ** 2)

mse_baseline = mse(data_with_sentiment['VIX Close'], garch_baseline.sigma2)
mse_sentiment = mse(data_with_sentiment['VIX Close'], garch_with_sentiment.sigma2)

print(f"MSE baseline: {mse_baseline:.3f}")
print(f"MSE with sentiment: {mse_sentiment:.3f}")

MSE baseline: 509949.092
MSE with sentiment: 511752.741


## Checking with arch library to make sure we are correct.

In [99]:
from arch import arch_model

model = arch_model(100*log_returns, vol='GARCH', mean='ARX', p=1, q=1)
garch_fit = model.fit(disp='off')

In [100]:
garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           AR - GARCH Model Results                           
==============================================================================
Dep. Variable:            log_returns   R-squared:                       0.000
Mean Model:                        AR   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1766.91
Distribution:                  Normal   AIC:                           3541.82
Method:            Maximum Likelihood   BIC:                           3562.27
                                        No. Observations:                 1229
Date:                Sat, Mar 01 2025   Df Residuals:                     1228
Time:                        05:23:16   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
Const          0.0974  2.528e-02      3.854  1.163e-04 [4.788e-02,  0.147]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0359  1.396e-02      2.569  1.020e-02 [8.504e-03,6.323e-02]
alpha[1]       0.1567  3.461e-02      4.527  5.980e-06   [8.886e-02,  0.225]
beta[1]        0.8226  3.520e-02     23.368 9.138e-121     [  0.754,  0.892]
============================================================================

Covariance estimator: robust
"""